# Steering Model Visualization
This code is a slightly modified version of the code produced by the comma.ai team. Original located here:
https://github.com/commaai/research/blob/master/view_steering_model.py

In [1]:
#Notes:

# len(test_images) <> len(df_truth) because model required predictions in batches of 128. Largest number divisible by 
# 128 is 5504

#!/usr/bin/env python
import pandas as pd
import os
import argparse
import sys
import numpy as np
import h5py
import pygame
import json
import matplotlib.pyplot as plt
import skimage
from skimage import io
import math

#Model "lookback" was 3 for LSTM layers
lookback = 3

pygame.init()
size = (320*2, 160*3)
pygame.display.set_caption("comma.ai data viewer")
screen = pygame.display.set_mode(size, pygame.DOUBLEBUF)

camera_surface = pygame.surface.Surface((320,240),0,24).convert()

#model = load_model(r'C:\Users\ryan\Desktop\Thinkful DS Sample Data - Main Course\Final Capstone\Final Model, Notebook, Test & Val Arrays\Final Model and Test Set\m51e.h5')

df_test = pd.DataFrame(np.load(r'C:\Users\ryan\Desktop\Thinkful DS Sample Data - Main Course\Final Capstone\Final Model, Notebook, Test & Val Arrays\Final Model and Test Set\test_preds.npy'))
df_test.columns = [['steering_angle']]
df_truth = pd.DataFrame(np.load(r'C:\Users\ryan\Desktop\Thinkful DS Sample Data - Main Course\Final Capstone\Final Model, Notebook, Test & Val Arrays\Final Model and Test Set\y_test.npy'))
df_truth.columns = [['steering_angle']]


c_steer_test = pd.read_csv(r'C:\Users\ryan\Desktop\Thinkful DS Sample Data - Main Course\Final Capstone\CH2_final_evaluation.csv')
test_root = os.path.normpath(r'C:\Users\ryan\Desktop\Thinkful DS Sample Data - Main Course\Final Capstone\center')


# ***** get perspective transform for images *****
from skimage import transform as tf

rsrc = \
 [[43.45456230828867, 118.00743250075844],
  [104.5055617352614, 69.46865203761757],
  [114.86050156739812, 60.83953551083698],
  [129.74572757609468, 50.48459567870026],
  [132.98164627363735, 46.38576532847949],
  [301.0336906326895, 98.16046448916306],
  [238.25686790036065, 62.56535881619311],
  [227.2547443287154, 56.30924933427718],
  [209.13359962247614, 46.817221154818526],
  [203.9561297064078, 43.5813024572758]]
rdst = \
 [[10.822125594094452, 1.42189132706374],
  [21.177065426231174, 1.5297552836484982],
  [25.275895776451954, 1.42189132706374],
  [36.062291434927694, 1.6376192402332563],
  [40.376849698318004, 1.42189132706374],
  [11.900765159942026, -2.1376192402332563],
  [22.25570499207874, -2.1376192402332563],
  [26.785991168638553, -2.029755283648498],
  [37.033067044190524, -2.029755283648498],
  [41.67121717733509, -2.029755283648498]]

tform3_img = tf.ProjectiveTransform()
tform3_img.estimate(np.array(rdst), np.array(rsrc))

def perspective_tform(x, y):
    p1, p2 = tform3_img((x,y))[0]
    return p2, p1

# ***** functions to draw lines *****
def draw_pt(img, x, y, color, sz=3):
    row, col = perspective_tform(x, y)
    #print('row', row)
    #print('col', col)
    #print(img.shape)
    #Had to reshape based on input image shape (RM)
    tmp = img.swapaxes(0,2).swapaxes(1,2)
    if row >= 0 and row < tmp.shape[0] and\
    col >= 0 and col < tmp.shape[1]:
        #Had to ensure integer values for rows and columns or it wouldn't work (RM)
        #Also added 75 and 160 pixels to row and column dimensions to properly center actual/prediction display lines
        tmp[math.floor(row+75)-sz:math.floor(row+75)+sz, math.floor(col+160)-sz:math.floor(col+160)+sz] = color

def draw_path(img, path_x, path_y, color):
    for x, y in zip(path_x, path_y):
        draw_pt(img, x, y, color)

# ***** functions to draw predicted path *****

def calc_curvature(v_ego, angle_steers, angle_offset=0):
    deg_to_rad = np.pi/180.
    slip_fator = 0.0014 # slip factor obtained from real data
    steer_ratio = 15.3  # from http://www.edmunds.com/acura/ilx/2016/road-test-specs/
    wheel_base = 2.67   # from http://www.edmunds.com/acura/ilx/2016/sedan/features-specs/

    angle_steers_rad = (angle_steers - angle_offset) #* deg_to_rad
    curvature = angle_steers_rad/(steer_ratio * wheel_base * (1. + slip_fator * v_ego**2))
    #Had to multiply by -1 to get direction correct (RM)
    return -1*curvature

def calc_lookahead_offset(v_ego, angle_steers, d_lookahead, angle_offset=0):
    #*** this function returns the lateral offset given the steering angle, speed and the lookahead distance
    curvature = calc_curvature(v_ego, angle_steers, angle_offset)

  # clip is to avoid arcsin NaNs due to too sharp turns
    y_actual = d_lookahead * np.tan(np.arcsin(np.clip(d_lookahead * curvature, -0.999, 0.999))/2.)
    return y_actual, curvature

def draw_path_on(img, speed_ms, angle_steers, color=(0,0,255)):
    path_x = np.arange(0., 50.1, 0.5)
    path_y, _ = calc_lookahead_offset(speed_ms, angle_steers, path_x)
    draw_path(img, path_x, path_y, color)

# ***** main loop *****
if __name__ == "__main__":
  #parser = argparse.ArgumentParser(description='Path viewer')
  #parser.add_argument('model', type=str, help='Path to model definition json. Model weights should be on the same path.')
  #parser.add_argument('--dataset', type=str, default="2016-06-08--11-46-01", help='Dataset/video clip name')
  #args = parser.parse_args()



  # default dataset is the validation data on the highway
  #dataset = args.dataset
  #skip = 300

  #log = h5py.File("dataset/log/"+dataset+".h5", "r")
  #cam = h5py.File("dataset/camera/"+dataset+".h5", "r")

  #print(log.keys())
    #Hard-coded speed rather than reading from some log file (RM)
    #speed_ms = 2 #log['speed'][i]  

#Iterate over test images
    for i in range(len(df_test)):
    #for i in range(1):
    #if i%100 == 0:
    #  print("%.2f seconds elapsed" % (i/100.0))
    #img = cam['X'][log['cam1_ptr'][i]].swapaxes(0,2).swapaxes(0,1)
        img = skimage.io.imread(test_root + '\\' + str(c_steer_test['frame_id'][i+lookback]) + '.jpg').swapaxes(0,2).swapaxes(0,1)
    #print('img', img.shape)
    #img = skimage.transform.resize(img, (img.shape[0] / (3/2), img.shape[1])).swapaxes(0,2).swapaxes(0,1)
    
        predicted_steers = df_test['steering_angle'].loc[i]
        angle_steers = df_truth['steering_angle'].loc[i]
    
        #Hard-coded speed rather than reading from some log file (RM)
        speed_ms = 0.2 #log['speed'][i]

        #Had to change division on angle steers from 10 to 3 to have it make visual sense on display (RM)
        draw_path_on(img, speed_ms, -angle_steers/3.0)
        draw_path_on(img, speed_ms, -predicted_steers/3.0, (0, 255, 0))

    # draw on
    #print('img', img.shape)
    #print('cam', camera_surface.shape)
        camera_surface_2x = pygame.transform.scale2x(camera_surface)
        pygame.surfarray.blit_array(camera_surface_2x, img.swapaxes(1,2))
    #camera_surface_2x = pygame.transform.scale2x(camera_surface)
        screen.blit(camera_surface_2x, (0,0))
        
        #Save images (only need to run this once to make an image set which we can then transform into a video)
        #pygame.image.save(camera_surface_2x, r'C:\Users\ryan\Desktop\Thinkful DS Sample Data - Main Course\Final Capstone\Final Model, Notebook, Test & Val Arrays\Img_Output\output_steer_image{}.png'.format(i))
        
        pygame.display.flip()
    # Do nothing in pygame window in response to mouse click (RM)
        for event in pygame.event.get():
            if event.type == pygame.MOUSEBUTTONUP:
                None 

In [2]:
#Now create a video file of all of those images
import cv2
import os

image_folder = r'C:\Users\ryan\Desktop\Thinkful DS Sample Data - Main Course\Final Capstone\Final Model, Notebook, Test & Val Arrays\Img_Output'
video_name =  r'C:\Users\ryan\Desktop\Thinkful DS Sample Data - Main Course\Final Capstone\Final Model, Notebook, Test & Val Arrays\Img_Output\output_video.mp4'

images = []
for i in range(len(df_test)):
    images.append('output_steer_image{}.png'.format(i))

# Determine the width and height from the first image
image_path = os.path.join(image_folder, images[0])
frame = cv2.imread(image_path)
cv2.imshow('video',frame)
height, width, channels = frame.shape

fourcc = cv2.VideoWriter_fourcc(*'MP4V')
video = cv2.VideoWriter(video_name, fourcc, 20.0, (width,height))

for image in images:

    image_path = os.path.join(image_folder, image)
    frame = cv2.imread(image_path)

    video.write(frame) # Write out frame to video

    cv2.imshow('video',frame)

# Release everything if job is finished
video.release()
cv2.destroyAllWindows()

# Check out the video!

https://youtu.be/c6e7YWUdXpI